In [1]:
# Setting up Imports 

import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Setting up Model Thresholds

model_memory_threshold = 60 # in KB

In [3]:
# Setting up Model Directories

models_dir = '../models/session1'
models = os.listdir(models_dir)
models

['2.keras', '0.keras', '1.keras']

In [4]:
# Get data

def get_data(input_shape):
    sample_nums = 1
    dims = [sample_nums]
    dims = dims + list(input_shape[:-1])
    data = tf.random.normal(shape = dims)
    data = (np.expand_dims(data, axis=-1))
    return data

In [5]:
# Model querying
results = []
num_samples = 100
for i in models:
    model = tf.keras.models.load_model(os.path.join(models_dir, i))
    input_dims = (model.layers[0].input_shape[0])[1:]
    data = get_data(input_dims)
    total = 0
    for i in range(num_samples):
        data = get_data(input_dims)
        tf.config.experimental.reset_memory_stats('GPU:0')
        result = tf.config.experimental.get_memory_info('GPU:0')
        curr = result["current"]
        model.predict(data, verbose=None)
        result = tf.config.experimental.get_memory_info('GPU:0')
        result = (result["peak"] - curr)/1000
        total = result + total
    result = total / num_samples
    results.append(result)
    print(f"Memory used:",result,"KB")
print(results)

Memory used: 139599.62368 KB
Memory used: 135731.14367999992 KB
Memory used: 135731.79903999984 KB


In [6]:
tf.config.experimental.reset_memory_stats('GPU:0')
tf.config.experimental.get_memory_info('GPU:0')

{'current': 5614080, 'peak': 5614080}